## Author:  Reshama Shaikh

## Geolocation Data
### Started:  Feb 8, 2022

NOTE:  
Want to get geo codes for participants for sprints


---

## Date

In [1]:
from datetime import date

current_date = date.today()
print("Today's date:", current_date)

Today's date: 2022-02-08


## Import libraries


In [2]:
import pandas as pd 
import numpy as np
import requests
import time
import csv
from pprint import pprint

In [3]:
import plotly.express as px
import plotly.graph_objects as go

In [4]:
%load_ext watermark
%watermark -n -v -m -g -iv

Python implementation: CPython
Python version       : 3.9.7
IPython version      : 7.25.0

Compiler    : Clang 11.1.0 
OS          : Darwin
Release     : 19.6.0
Machine     : x86_64
Processor   : i386
CPU cores   : 4
Architecture: 64bit

Git hash: 9c164f2cb0ac0ae325f5ccb2cc6a2724871ac740

csv     : 1.0
requests: 2.26.0
pandas  : 1.3.3
plotly  : 5.4.0
numpy   : 1.20.2



## Set pandas view options

In [5]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Read in csv file

In [6]:
path_datain = "../data/"
path_graphs = "../graphs/"

In [7]:
df_use = pd.read_csv(path_datain+'/afme2_people.csv')

In [9]:
df_use['location'] = df_use.apply(lambda x:'%s %s' % (x['city'],x['country']),axis=1)

In [10]:
df_use.shape

(74, 32)

In [13]:
df_use.head(2)

,Timestamp,Attended,status,city,province_state,country,gender,pronouns,join_sprint,role,os_system,student_school,student_field,company,are_you_urp,learn_of_sprint,contributor_status,prior_sprints,os_experience,prior_os,prior_pr,git,prog_langs,python_experience,used_sklearn,scikit_repo_familiar,why_in_sprint,can_do_postwork,can_do_prepwork,internet_data,primary_spoken_language,location
0,8/28/2021 6:59:19,did not RSVP,0,KINSHASA,KINSHASA,Congo,Man,he/him,"Yes, Saturday, Oct 23, 2021 (5pm to 9pm, +3 EA...","Analyst, Researcher",Windows,NaN,NaN,Digital Security Group,"Yes, I identify as an under-represented person.","Slack, Twitter",new,"No, never.",Advanced,"Yes, writing technical documentation, Yes, wri...",No,"used Git, but am not comfortable","Python, R, MATLAB, C+ / C++",Advanced,I am not at all familiar with scikit-learn.,I am not at all familiar with scikit-learn.,I will like to learn and build my capacity dur...,Yes,Yes,Yes,French,KINSHASA Congo
1,8/28/2021 10:24:22,1,1,Gdansk,Pomorskie,Poland,Man,he/him,"Yes, Saturday, Oct 23, 2021 (5pm to 9pm, +3 EA...",Data Scientist,"Linux, Windows, Cloud",NaN,NaN,NaN,"Yes, I identify as an under-represented person.",NumFOCUS newsletter,new,"No, never.",Beginner,"Yes, writing blogs, Yes, answering user questi...","Yes, some minor updates to fastai, icevision a...","comfortable submitting a PR, comfortable setti...",Python,Intermediate,I have used some scikit-learn for machine lear...,"I am familiar, I have looked at the codebase.",Contribute to OpenSourse and Scikit learn!,Yes,Yes,No,"Polish, English",Gdansk Poland


---

In [14]:
from opencage.geocoder import OpenCageGeocode

https://amaral.northwestern.edu/blog/getting-long-lat-list-cities
#!pip install opencage

In [15]:
# log in using GH authentication
# https://opencagedata.com/dashboard#geocoding

key = "aa82453ae4e44055a43dd954ce8c4bef"  # get api key from:  https://opencagedata.com
key = "2af537650a554a2b80705c9aeb45f8a6"
geocoder = OpenCageGeocode(key)


In [16]:
loc_codes =[]

def get_code(place, logprint):
    query=place
    results = geocoder.geocode(query)
    #pprint (results)

    lat = results[0]['geometry']['lat']
    lng = results[0]['geometry']['lng']

    continent = results[0]['components']['continent']
    country = results[0]['components']['country']
    #state = results[0]['components']['state']
    #city = results[0]['components']['city']
    iso_alpha = results[0]['annotations']['currency']['iso_code']
    iso_num = results[0]['annotations']['currency']['iso_numeric']
    
    #temp = [place, continent, country,  city, lat, lng]
    temp = [place, continent, country, iso_alpha, iso_num,  lat, lng]

    loc_codes.append(temp)
    if logprint==1:
        print('-'*10)
        pprint (results)
        print('-'*10)
        print(place)
        print(continent)
        print(country)
        #print(state)
        #print(city)
        print(lat,lng)
        print('-'*10)
        #print(iso_alpha, iso_num)
    print(temp)
    return temp


In [17]:
df_loc=df_use.copy()

count=-1
loc_list = list()

for i, row in df_loc.iterrows():
    max_rows = 300
    #max_rows = 10
    if count <= max_rows:
        count+=1
        #print(count)
        #print(f"Index: {i}", f"{row['location']}", f"{row['location2']}")
        temp = row['location']
        #print(temp)
        temp2 = get_code(temp, 0)
        #print(temp2)
        loc_list.append(temp2)
        #print(f"{row['city']}")
        
    if count % 10 ==0:
        print(count)

['KINSHASA Congo', 'Africa', 'Democratic Republic of the Congo', 'CDF', '976', -4.3983689, 15.4035782]
0
['Gdansk Poland', 'Europe', 'Poland', 'PLN', '985', 54.3611929, 18.6286088]
['Sofia Bulgaria', 'Europe', 'Bulgaria', 'BGN', '975', 42.6978634, 23.3221789]
['Nairobi Kenya', 'Africa', 'Kenya', 'KES', '404', -1.2832533, 36.8172449]
['Lagos Nigeria', 'Africa', 'Nigeria', 'NGN', '566', 6.4550575, 3.3941795]
['Nairobi Kenya', 'Africa', 'Kenya', 'KES', '404', -1.2832533, 36.8172449]
['Nairobi, Kenya Kenya', 'Africa', 'Kenya', 'KES', '404', -1.3038576, 36.8248525]
['Nairobi Kenya', 'Africa', 'Kenya', 'KES', '404', -1.2832533, 36.8172449]
['Nairobi Kenya', 'Africa', 'Kenya', 'KES', '404', -1.2832533, 36.8172449]
['Cairo Egypt', 'Africa', 'Egypt', 'EGP', '818', 30.0443879, 31.2357257]
['Lagos Nigeria', 'Africa', 'Nigeria', 'NGN', '566', 6.4550575, 3.3941795]
10
['Abeokuta Nigeria', 'Africa', 'Nigeria', 'NGN', '566', 7.1547129, 3.3467344]
['Ikorodu Nigeria', 'Africa', 'Nigeria', 'NGN', '566',

In [50]:
type(loc_list)

list

In [51]:
import csv

fields = ['location', 'continent_o', 'country_o', 'iso_alpha', 'iso_num',  'lat', 'lng']

with open('../data/afme_geo_2022-02.csv', 'w') as f:
      
    # using csv.writer method from CSV package
    write = csv.writer(f)
      
    write.writerow(fields)
    write.writerows(loc_list)

In [81]:
len(loc_list)

74

In [82]:
#print("\nloc_list")
#pprint(loc_list[0:3]) 

In [83]:
from pandas import DataFrame

df_locinfo = DataFrame(loc_list, columns=['location', 'continent_o', 'country_o', \
                                          'iso_alpha', 'iso_num',  'lat', 'lng'])

In [84]:
len(df_locinfo)

74

In [85]:
df_locinfo2 = df_locinfo.drop_duplicates(subset=['location'])

In [86]:
len(df_locinfo2)

54

In [87]:
df_locinfo.head(3)

,location,continent_o,country_o,iso_alpha,iso_num,lat,lng
0,KINSHASA Congo,Africa,Democratic Republic of the Congo,CDF,976,-4.398369,15.403578
1,Gdansk Poland,Europe,Poland,PLN,985,54.361193,18.628609
2,Sofia Bulgaria,Europe,Bulgaria,BGN,975,42.697863,23.322179


In [88]:
df_use.head(3)

,Timestamp,Attended,status,city,province_state,country,gender,pronouns,join_sprint,role,os_system,student_school,student_field,company,are_you_urp,learn_of_sprint,contributor_status,prior_sprints,os_experience,prior_os,prior_pr,git,prog_langs,python_experience,used_sklearn,scikit_repo_familiar,why_in_sprint,can_do_postwork,can_do_prepwork,internet_data,primary_spoken_language,location
0,8/28/2021 6:59:19,did not RSVP,0,KINSHASA,KINSHASA,Congo,Man,he/him,"Yes, Saturday, Oct 23, 2021 (5pm to 9pm, +3 EA...","Analyst, Researcher",Windows,NaN,NaN,Digital Security Group,"Yes, I identify as an under-represented person.","Slack, Twitter",new,"No, never.",Advanced,"Yes, writing technical documentation, Yes, wri...",No,"used Git, but am not comfortable","Python, R, MATLAB, C+ / C++",Advanced,I am not at all familiar with scikit-learn.,I am not at all familiar with scikit-learn.,I will like to learn and build my capacity dur...,Yes,Yes,Yes,French,KINSHASA Congo
1,8/28/2021 10:24:22,1,1,Gdansk,Pomorskie,Poland,Man,he/him,"Yes, Saturday, Oct 23, 2021 (5pm to 9pm, +3 EA...",Data Scientist,"Linux, Windows, Cloud",NaN,NaN,NaN,"Yes, I identify as an under-represented person.",NumFOCUS newsletter,new,"No, never.",Beginner,"Yes, writing blogs, Yes, answering user questi...","Yes, some minor updates to fastai, icevision a...","comfortable submitting a PR, comfortable setti...",Python,Intermediate,I have used some scikit-learn for machine lear...,"I am familiar, I have looked at the codebase.",Contribute to OpenSourse and Scikit learn!,Yes,Yes,No,"Polish, English",Gdansk Poland
2,9/9/2021 10:16:34,1,1,Sofia,Sofia,Bulgaria,Man,he/him,"Yes, Saturday, Oct 23, 2021 (5pm to 9pm, +3 EA...",Data Scientist,Mac,Not a student,Not a student,Financial Times,"No, I do not identify as an under-represented ...",by invitation,returning,"Yes, in June 2020 (Online) (Global)",Beginner,"Yes, submitted a PR to a code repository, Yes,...",Yes,"comfortable submitting a PR, comfortable setti...","Python, R",Intermediate,I have used some scikit-learn for machine lear...,"I am familiar, I have submitted a PR in the past","Gain experience, contribute to a great library",Yes,Yes,No,"Bulgarian, English",Sofia Bulgaria


In [89]:
df_locinfo2 = pd.merge(df_locinfo2, df_use, on=['location'], how='left')

In [90]:
len(df_locinfo2)

74

In [91]:
df_locinfo2

,location,continent_o,country_o,iso_alpha,iso_num,lat,lng,Timestamp,Attended,status,city,province_state,country,gender,pronouns,join_sprint,role,os_system,student_school,student_field,company,are_you_urp,learn_of_sprint,contributor_status,prior_sprints,os_experience,prior_os,prior_pr,git,prog_langs,python_experience,used_sklearn,scikit_repo_familiar,why_in_sprint,can_do_postwork,can_do_prepwork,internet_data,primary_spoken_language
0,KINSHASA Congo,Africa,Democratic Republic of the Congo,CDF,976,-4.398369,15.403578,8/28/2021 6:59:19,did not RSVP,0,KINSHASA,KINSHASA,Congo,Man,he/him,"Yes, Saturday, Oct 23, 2021 (5pm to 9pm, +3 EA...","Analyst, Researcher",Windows,NaN,NaN,Digital Security Group,"Yes, I identify as an under-represented person.","Slack, Twitter",new,"No, never.",Advanced,"Yes, writing technical documentation, Yes, wri...",No,"used Git, but am not comfortable","Python, R, MATLAB, C+ / C++",Advanced,I am not at all familiar with scikit-learn.,I am not at all familiar with scikit-learn.,I will like to learn and build my capacity dur...,Yes,Yes,Yes,French
1,Gdansk Poland,Europe,Poland,PLN,985,54.361193,18.628609,8/28/2021 10:24:22,1,1,Gdansk,Pomorskie,Poland,Man,he/him,"Yes, Saturday, Oct 23, 2021 (5pm to 9pm, +3 EA...",Data Scientist,"Linux, Windows, Cloud",NaN,NaN,NaN,"Yes, I identify as an under-represented person.",NumFOCUS newsletter,new,"No, never.",Beginner,"Yes, writing blogs, Yes, answering user questi...","Yes, some minor updates to fastai, icevision a...","comfortable submitting a PR, comfortable setti...",Python,Intermediate,I have used some scikit-learn for machine lear...,"I am familiar, I have looked at the codebase.",Contribute to OpenSourse and Scikit learn!,Yes,Yes,No,"Polish, English"
2,Sofia Bulgaria,Europe,Bulgaria,BGN,975,42.697863,23.322179,9/9/2021 10:16:34,1,1,Sofia,Sofia,Bulgaria,Man,he/him,"Yes, Saturday, Oct 23, 2021 (5pm to 9pm, +3 EA...",Data Scientist,Mac,Not a student,Not a student,Financial Times,"No, I do not identify as an under-represented ...",by invitation,returning,"Yes, in June 2020 (Online) (Global)",Beginner,"Yes, submitted a PR to a code repository, Yes,...",Yes,"comfortable submitting a PR, comfortable setti...","Python, R",Intermediate,I have used some scikit-learn for machine lear...,"I am familiar, I have submitted a PR in the past","Gain experience, contribute to a great library",Yes,Yes,No,"Bulgarian, English"
3,Sofia Bulgaria,Europe,Bulgaria,BGN,975,42.697863,23.322179,10/5/2021 4:16:02,1,1,Sofia,Sofia - City,Bulgaria,Woman,she/her,"Yes, Saturday, Oct 23, 2021 (5pm to 9pm, +3 EA...",Data Scientist,Mac,NaN,NaN,Financial Times,"No, I do not identify as an under-represented ...","a friend, colleague, etc. (word of mouth)",new,"No, never.",No experience,No,No,"comfortable submitting a PR, comfortable setti...","Python, R, SAS, SQL",Beginner,I have used some scikit-learn for machine lear...,I have never looked at the source code for sci...,I would like to contribute to OS as well as up...,Not sure I cannot commit to it.,Yes,No,"English, Bulgarian"
4,Nairobi Kenya,Africa,Kenya,KES,404,-1.283253,36.817245,9/9/2021 12:10:54,1,1,Nairobi,Nairobi,Kenya,Woman,she/her,"Yes, Saturday, Oct 23, 2021 (5pm to 9pm, +3 EA...","Statistician, Student",Windows,University of Nairobi,MSc. Social Statistics,NaN,"No, I do not identify as an under-represented ...",by invitation,returning,"Yes, in June 2020 (Online) (Global), Yes, in ...",Intermediate,"Yes, submitted a PR to a code repository, Yes,...",Yes,"comfortable submitting a PR, comfortable cloni...","Python, R",Intermediate,I have used some scikit-learn for machine lear...,"I am familiar, I have submitted a PR in the past",To contribute towards solving more issues,Yes,Yes,NaN,"English, Kiswahili"
5,Nairobi Kenya,Africa,Kenya,KES,404,-1.283253,36.817245,9/10/2021 2:00:27,1,1,Nairobi,Kajiado North/Rift Valley,Kenya,Man,he/him,"Yes, Saturday, Oct 23, 2021 (5pm to 9pm, +3 EA...","Data Scientist, Researcher, computational micr...","Linux, Cloud

In [92]:
df_locinfo2 = df_locinfo2.rename(columns={'province_state': 'state-province',
                        'Timestamp': 'timestamp',
                        'Attended' : 'attendance_status',
                       })

In [95]:
# Write out the dataframe to a file

df_locinfo2.to_csv('../data/derived_geocodes.csv', index=False)

In [97]:
# Read csv file into a dataframe

#df_location = pd.read_csv('../data/derived_geocodes.csv', index_col=False)  

In [98]:
#df_location.head(3)

,location,continent_o,country_o,iso_alpha,iso_num,lat,lng,timestamp,attendance_status,status,city,state-province,country,gender,pronouns,join_sprint,role,os_system,student_school,student_field,company,are_you_urp,learn_of_sprint,contributor_status,prior_sprints,os_experience,prior_os,prior_pr,git,prog_langs,python_experience,used_sklearn,scikit_repo_familiar,why_in_sprint,can_do_postwork,can_do_prepwork,internet_data,primary_spoken_language,text
0,KINSHASA Congo,Africa,Democratic Republic of the Congo,CDF,976,-4.398369,15.403578,8/28/2021 6:59:19,did not RSVP,0,KINSHASA,KINSHASA,Congo,Man,he/him,"Yes, Saturday, Oct 23, 2021 (5pm to 9pm, +3 EA...","Analyst, Researcher",Windows,NaN,NaN,Digital Security Group,"Yes, I identify as an under-represented person.","Slack, Twitter",new,"No, never.",Advanced,"Yes, writing technical documentation, Yes, wri...",No,"used Git, but am not comfortable","Python, R, MATLAB, C+ / C++",Advanced,I am not at all familiar with scikit-learn.,I am not at all familiar with scikit-learn.,I will like to learn and build my capacity dur...,Yes,Yes,Yes,French,"KINSHASA,"
1,Gdansk Poland,Europe,Poland,PLN,985,54.361193,18.628609,8/28/2021 10:24:22,1,1,Gdansk,Pomorskie,Poland,Man,he/him,"Yes, Saturday, Oct 23, 2021 (5pm to 9pm, +3 EA...",Data Scientist,"Linux, Windows, Cloud",NaN,NaN,NaN,"Yes, I identify as an under-represented person.",NumFOCUS newsletter,new,"No, never.",Beginner,"Yes, writing blogs, Yes, answering user questi...","Yes, some minor updates to fastai, icevision a...","comfortable submitting a PR, comfortable setti...",Python,Intermediate,I have used some scikit-learn for machine lear...,"I am familiar, I have looked at the codebase.",Contribute to OpenSourse and Scikit learn!,Yes,Yes,No,"Polish, English","Gdansk,"
2,Sofia Bulgaria,Europe,Bulgaria,BGN,975,42.697863,23.322179,9/9/2021 10:16:34,1,1,Sofia,Sofia,Bulgaria,Man,he/him,"Yes, Saturday, Oct 23, 2021 (5pm to 9pm, +3 EA...",Data Scientist,Mac,Not a student,Not a student,Financial Times,"No, I do not identify as an under-represented ...",by invitation,returning,"Yes, in June 2020 (Online) (Global)",Beginner,"Yes, submitted a PR to a code repository, Yes,...",Yes,"comfortable submitting a PR, comfortable setti...","Python, R",Intermediate,I have used some scikit-learn for machine lear...,"I am familiar, I have submitted a PR in the past","Gain experience, contribute to a great library",Yes,Yes,No,"Bulgarian, English","Sofia,"


In [66]:
current_date = date.today()
print("Today's date:", current_date)

Today's date: 2022-02-08


In [ ]:
print x

---

In [ ]:
loc_codes =[]

def get_code(place, logprint):
    query=place
    results = geocoder.geocode(query)
    #pprint (results)

    lat = results[0]['geometry']['lat']
    lng = results[0]['geometry']['lng']

    continent = results[0]['components']['continent']
    country = results[0]['components']['country']
    #state = results[0]['components']['state']
    #city = results[0]['components']['city']
    iso_alpha = results[0]['annotations']['currency']['iso_code']
    iso_num = results[0]['annotations']['currency']['iso_numeric']
    
    #temp = [place, continent, country,  city, lat, lng]
    temp = [place, continent, country, iso_alpha, iso_num,  lat, lng]

    loc_codes.append(temp)
    if logprint==1:
        print('-'*10)
        pprint (results)
        print('-'*10)
        print(place)
        print(continent)
        print(country)
        #print(state)
        #print(city)
        print(lat,lng)
        print('-'*10)
        #print(iso_alpha, iso_num)
    return temp

#get_code("Bijuesca, Spain", 0)
#get_code("Dublin Ireland", 0)
#get_code("Chicago Illinois", 0)
#get_code("New York", 1)
#get_code("Akure Nigeria",1)
#get_code("Aracati South, America",1)
#get_code("Arlington Texas USA", 1)
#get_code("Brazzaville Congo", 1)
#get_code("Manila Philippines", 1)
#get_code("Zimbabwe", 1)
#get_code("Guayaquil Ecuador", 1)
#get_code("Quito Ecuador", 1)

#get_code("Mérida Venezuela", 1)

#get_code("Merida Venezuela", 1)



pprint(loc_codes)

In [ ]:
type(loc_codes)

In [ ]:
# from pandas import DataFrame

# df_loc = DataFrame(loc_codes,columns=['place','continent','country','state','city','lat','lng'])

In [ ]:
df_loc=df_use.copy()

# delete some records temporarily

index_names = df_loc[df_loc['location'] == 'Harare Zimbabwe'].index 
df_loc.drop(index_names, inplace = True) 

index_names = df_loc[df_loc['location'] == 'Remote Global'].index 
df_loc.drop(index_names, inplace = True)

index_names = df_loc[df_loc['location'] == 'Guayaquil Ecuador'].index 
df_loc.drop(index_names, inplace = True)

index_names = df_loc[df_loc['location'] == 'Quito Ecuador'].index 
df_loc.drop(index_names, inplace = True)


index_names = df_loc[df_loc['location'] == 'Mérida Venezuela'].index 
df_loc.drop(index_names, inplace = True)

In [ ]:
count=-1
loc_list = list()

for i, row in df_loc.iterrows():
    max_rows = 300
    max_rows = 10
    if count <= max_rows:
        count+=1
        #print(count)
        #print(f"Index: {i}", f"{row['location']}", f"{row['location2']}")
        temp = row['location']
        #print(temp)
        temp2 = get_code(temp, 0)
        #print(temp2)
        loc_list.append(temp2)
        #print(f"{row['city']}")
        
    if count % 30 ==0:
        print(count)


---